In [ ]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.conda/envs/nvflare/bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

In [ ]:
%%bash
# 請修改下四個項目
node_hostname=$(hostname -s)
node_admin=admin@nvidia.com
node_server_port=8002
node_admin_port=8003

cat <<EOF > yml/demo.yml
api_version: 3
name: example_project
description: NVIDIA FLARE sample project yaml file

participants:
  # Change the name of the server (server1) to the Fully Qualified Domain Name
  # (FQDN) of the server, for example: server1.example.com.
  # Ensure that the FQDN is correctly mapped in the /etc/hosts file.
  - name: ${node_hostname}
    type: server
    org: nvidia
    fed_learn_port: ${node_server_port}
    admin_port: ${node_admin_port}
  - name: site-1
    type: client
    org: nvidia
    # Specifying listening_host will enable the creation of one pair of
    # certificate/private key for this client, allowing the client to function
    # as a server for 3rd-party integration.
    # The value must be a hostname that the external trainer can reach via the network.
    # listening_host: site-1-lh
  - name: site-2
    type: client
    org: nvidia
  - name: ${node_admin}
    type: admin
    org: nvidia
    role: project_admin

# The same methods in all builders are called in their order defined in builders section
builders:
  - path: nvflare.lighter.impl.workspace.WorkspaceBuilder
    args:
      template_file:
        - master_template.yml
        - aws_template.yml
        - azure_template.yml
  - path: nvflare.lighter.impl.template.TemplateBuilder
  - path: nvflare.lighter.impl.static_file.StaticFileBuilder
    args:
      # config_folder can be set to inform NVIDIA FLARE where to get configuration
      config_folder: config

      # scheme for communication driver (currently supporting the default, grpc, only).
      # scheme: grpc

      # app_validator is used to verify if uploaded app has proper structures
      # if not set, no app_validator is included in fed_server.json
      # app_validator: PATH_TO_YOUR_OWN_APP_VALIDATOR

      # when docker_image is set to a docker image name, docker.sh will be generated on server/client/admin
      # docker_image:

      # download_job_url is set to http://download.server.com/ as default in fed_server.json.  You can override this
      # to different url.
      # download_job_url: http://download.server.com/

      overseer_agent:
        path: nvflare.ha.dummy_overseer_agent.DummyOverseerAgent
        # if overseer_exists is true, args here are ignored.  Provisioning
        #   tool will fill role, name and other local parameters automatically.
        # if overseer_exists is false, args in this section will be used and the sp_end_point
        # must match the server defined above in the format of SERVER_NAME:FL_PORT:ADMIN_PORT
        # 
        overseer_exists: false
        args:
          sp_end_point: ${node_hostname}:${node_server_port}:${node_admin_port}

  - path: nvflare.lighter.impl.cert.CertBuilder
  - path: nvflare.lighter.impl.signature.SignatureBuilder

EOF

In [ ]:
# 移除工作項目dmeo1 以及建立工作項目dmeo1 
!rm -rf workspace/demo01
!nvflare provision -p yml/demo.yml -w workspace/demo01
!sed -i "s#/tmp/nvflare/#/tmp/nvflare_$(whoami)/#g"  workspace/demo01/example_project/prod_00/$(hostname -s)/local/resources.json.default

In [ ]:
# 下載特定版本 NVFLARE
import os
tmp=!echo $(hostname)
HOSTNAME=tmp[0]
os.environ['http_proxy'] = "socks5:/"+HOSTNAME+":12345" 
os.environ['https_proxy'] = "socks5://"+HOSTNAME+":12345" 
!git clone --branch 2.3 https://github.com/NVIDIA/NVFlare.git NVFlare-2.3

In [ ]:
# 複製工作項目
!rm -rf workspace/demo01/example_project/prod_00/admin@nvidia.com/transfer/*
!cp -rf NVFlare-2.3/examples/hello-world/hello-numpy-cross-val/jobs/hello-numpy-cross-val \
workspace/demo01/example_project/prod_00/admin@nvidia.com/transfer/
!cp -rf NVFlare-2.3/examples/hello-world/hello-numpy-sag/jobs/hello-numpy-sag \
workspace/demo01/example_project/prod_00/admin@nvidia.com/transfer/

In [ ]:
# 移除目前正在執行的nvflare
!ps -efww |grep nvflare/workspace   | awk '{print $2}' | xargs kill -9 

### 請於terminal 下依序啟動以下內容
#### 1. nvflare server
```bash
ml tools/miniconda3 
conda activate nvflare
./workspace/demo01/example_project/prod_00/$(hostname -s)/startup/start.sh
```

### 請於terminal 下依序啟動以下內容
#### 2. nvflare site-1
```bash
ml tools/miniconda3 
conda activate nvflare
./workspace/demo01/example_project/prod_00/site-1/startup/start.sh
```

### 請於terminal 下依序啟動以下內容
#### 3. nvflare site-2
```bash
ml tools/miniconda3 
conda activate nvflare
./workspace/demo01/example_project/prod_00/site-2/startup/start.sh
```

### 請於terminal 下依序啟動以下內容
### 4. nvflare admin
```bash
ml tools/miniconda3 
conda activate nvflare
./workspace/demo01/example_project/prod_00/admin@nvidia.com/startup/fl_admin.sh
```

### 請於terminal 下依序啟動以下內容
### 5. nvflare admin
```bash
?
check_status client
check_status server
list_jobs
submit_job hello-numpy-sag
submit_job hello-numpy-cross-val
list_jobs
download_job b8cd4fc1-cd9b-48ee-a22e-1060b74be57b
download_job b8cd4fc1-cd9b-48ee-a22e-1060b74be57b /tmp/demo
remove_client site-1
remove_client site-2
shutdown client site-1
shutdown client site-2
shutdown server
```